In [49]:
import numpy as np

%matplotlib ipympl
%matplotlib inline
import matplotlib.pyplot as plt
from astropy.table import Table

I don't understand why you need to run this command twice to get an interactive matplotlib display

In [50]:
%matplotlib ipympl

In [51]:
import lsst.daf.persistence        as dafPersist
import lsst.afw.geom               as afwGeom
from matplotlib.colors import LogNorm
import lsst.afw.table              as afwTable
import lsst.meas.extensions.psfex.psfexPsfDeterminer

import lsst.afw.display            as afwDisplay
import lsst.afw.display.utils as afwDisplayUtils

#### Load the high-level "tasks" that process the pixels

In [52]:
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask

#### Setup the displays (by default an interface to ds9, which won't work in the LSP)

In [53]:
afwDisplay.setDefaultBackend("matplotlib")

## Create the task

In [54]:
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
config.measurePsf.psfDeterminer.name = "psfex"
charImageTask = CharacterizeImageTask(None, config=config)

## Time to process some data

#### Read the input data

In [90]:
butler = dafPersist.Butler("/Users/naka.keigo/Desktop/integ/INTEGRATION/rerun/integration/arc/")

In [91]:
dataId = dict(visit=31)
exposure = butler.get("calexp", visit=31, arm="r",spectrograph=1)
dataRef=butler.dataRef("calexp", visit=31, arm="r", spectrograph=1)


In [92]:
degrees = lsst.geom.degrees
cdMatrix = np.array([[1.0e-4, 0.0], [0.0, 1.0e-4]], dtype=float)
exposure.setWcs(lsst.afw.geom.makeSkyWcs(crval=lsst.geom.SpherePoint(0*degrees, 0*degrees),
                                            crpix=lsst.geom.Point2D(0.0, 0.0),
                                            cdMatrix=cdMatrix))

In [98]:
%matplotlib ipympl

#disp = afwDisplay.Display(reopenPlot=True)
disp = afwDisplay.Display(reopenPlot=False, fastMaskDisplay=True)
disp.scale('asinh', -20, 30, Q=8)
disp.mtv(exposure, title=dataId)

disp.zoom(8, 500, 1500);

FigureCanvasNbAgg()

### Process the pixels

##### Characterise the exposure (e.g. estimate the PSF)

In [59]:
from lsst.obs.base import ExposureIdInfo
result = charImageTask.detectMeasureAndEstimatePsf(exposure,ExposureIdInfo(), None)
#psf = exposure.getPsf()

In [60]:
#result = charImageTask.run(dataRef, exposure, doUnpersist=False)
#result = charImageTask.run(exposure)

In [61]:
maskedImage=exposure.getMaskedImage()
psf = exposure.getPsf()
w, h = exposure.getDimensions()

In [73]:
%matplotlib ipympl

mos = afwDisplayUtils.Mosaic()

for y in np.linspace(0, h, 10):
    for x in np.linspace(0, w, 10):
        im = psf.computeImage(afwGeom.PointD(x, y)).convertF()
        mos.append(im)

disp2 = afwDisplay.Display(2, reopenPlot=True)
disp2.scale('asinh', 'zscale')
mos.makeMosaic(display=disp2);

FigureCanvasNbAgg()

In [99]:
from scipy.ndimage import label

mask1=exposure.getMask().array
mask2=(mask1==32) *mask1

values = np.unique(mask2.ravel())
result = np.zeros_like(mask2)
offset = 0

for v in values:
    labeled, num_features = label(mask2 == v)
    result += labeled + offset*(labeled > 0)
    offset += num_features

plt.imshow(result)

In [89]:
i=82

py = int(round(np.where(result==i)[1].mean()))
px= int(round(np.where(result==i)[0].mean()))

print(px,py)


box=exposure.getImage().array[px-20:px+21,py-20:py+21]
psfplt= psf.computeImage(afwGeom.PointD(py, px)).convertF()

normbox = (box)/box.max()
normpsf = (psfplt.array)/psfplt.array.max()

sigma = np.sqrt((normbox-normpsf)* (normbox-normpsf))

    #normbox = (box-box.min())/(box.max()-box.min())
    #normpsf = (psfplt.array-psfplt.array.min())/(psfplt.array.max()-psfplt.array.min())

    #print(normbox)
    #print(normpsf)


plt.figure(figsize=(6,5))
#plt.imshow(box, origin='lower', extent=tlim+flim, aspect='auto')
plt.imshow(psfplt.array,extent=tlim+flim,)

plt.figure(figsize=(6,5))
#plt.imshow(box, origin='lower', extent=tlim+flim, aspect='auto')
plt.imshow(box,extent=tlim+flim,)

plt.figure(figsize=(6,5))
plt.imshow(sigma, extent=tlim+flim)


424 2015


FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [85]:
import matplotlib.gridspec as gridspe

plt.figure(figsize=(6,5))
plt.plot(normpsf.mean(1))
plt.plot(normbox.mean(1))

plt.figure(figsize=(6,5))
plt.plot(normpsf.mean(0))
plt.plot(normbox.mean(0))


FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [71]:
plt.figure(figsize=(6,5))
#plt.xlim(-0.05,0.05)

px = []
py = []
sigma = []

for i in range(32,offset): 
#for i in range(32,202): 
    py.append(int(round(np.where(result==i)[1].mean())))
    px.append(int(round(np.where(result==i)[0].mean())))

    iy = int(np.where(result==i)[1].mean())
    ix = int(np.where(result==i)[0].mean())

    lengthx=np.where(result==i)[0].max()-np.where(result==i)[0].min()
    lengthy=np.where(result==i)[1].max()-np.where(result==i)[1].min()


    #print(i, lengthx,lengthy)
    
    box=exposure.getImage().array[ix-20:ix+21,iy-20:iy+21]
    psfplt= psf.computeImage(afwGeom.PointD(iy, ix)).convertF()

    normbox = (box)/box.max()
    normpsf = (psfplt.array)/psfplt.array.max()

    #normbox = (box-box.min())/(box.max()-box.min())
    #normpsf = (psfplt.array-psfplt.array.min())/(psfplt.array.max()-psfplt.array.min())
    
    if lengthx > 15 or lengthy>15:
        sigma.append(0)
        #print("failed")
        continue
    else:
         sigma.append(np.sqrt((normbox-normpsf)* (normbox-normpsf)).mean())


    
plt.scatter(py, px, c=sigma, cmap='Blues')    
plt.colorbar()

FigureCanvasNbAgg()

KeyboardInterrupt: 

In [564]:
plt.figure(figsize=(6,5))
plt.hist(sigma,40)    

FigureCanvasNbAgg()

(array([ 121.,    4.,    4.,   15.,   17.,   25.,   27.,   34.,   28.,
          19.,   12.,   13.,   15.,    8.,   13.,   13.,   12.,    9.,
           8.,    7.,    8.,   10.,    6.,   12.,   12.,    9.,    8.,
          13.,    5.,    7.,    6.,    5.,    9.,    2.,    1.,    3.,
           2.,    0.,    1.,    4.]),
 array([ 0.        ,  0.000549  ,  0.001098  ,  0.001647  ,  0.00219599,
         0.00274499,  0.00329399,  0.00384299,  0.00439199,  0.00494099,
         0.00548998,  0.00603898,  0.00658798,  0.00713698,  0.00768598,
         0.00823498,  0.00878397,  0.00933297,  0.00988197,  0.01043097,
         0.01097997,  0.01152897,  0.01207796,  0.01262696,  0.01317596,
         0.01372496,  0.01427396,  0.01482296,  0.01537195,  0.01592095,
         0.01646995,  0.01701895,  0.01756795,  0.01811695,  0.01866594,
         0.01921494,  0.01976394,  0.02031294,  0.02086194,  0.02141094,
         0.02195993]),
 <a list of 40 Patch objects>)